In [1]:
import tensorflow as tf
import numpy as np

In [2]:
f = open("C:/data/hscode/train/hscode_ulocal_test_gooddesc.txt", 'r')
g = open("C:/data/hscode/train/hscode_ulocal_test_hscode.txt", 'r')

In [3]:
input_lines = f.readlines()
output_lines = g.readlines()

In [4]:
input_char_sequence = list("".join(input_lines))
input_char_sequence = list(set(input_char_sequence))
#input_char_sequence.remove('\n')
print(input_char_sequence)

['1', 'R', 'N', 'D', ',', '(', 'I', '%', 'S', 'K', 'T', 'W', '-', 'E', '2', '[', '\t', '8', 'Y', '=', "'", '5', '+', ' ', '"', 'M', '&', 'U', 'O', 'Z', 'X', 'V', '7', '9', 'B', '\n', '_', 'P', 'H', 'Q', ']', ':', '>', '`', '#', '/', '!', '6', 'F', '0', 'C', 'G', '3', '.', '$', ';', 'A', 'J', '*', '<', ')', '4', '?', 'L']


In [5]:
output_char_sequence = list("".join(output_lines))
output_char_sequence = list(set(output_char_sequence))
#output_char_sequence.remove('\n')
print(output_char_sequence)

['0', '1', '5', '9', '7', ' ', '3', '\n', '4', '2', '8', '6']


In [6]:
char_sequence = input_char_sequence + output_char_sequence
char_sequence += ['^', '$']
char_sequence = list(set(char_sequence))
char_sequence.remove('\n')
print(char_sequence)

['1', 'R', 'N', '(', ',', 'D', 'I', '%', 'S', 'K', 'T', 'W', '-', 'E', '2', '[', '\t', '=', '8', 'Y', "'", '5', '^', 'L', '+', ' ', '"', 'M', '&', 'U', 'O', 'Z', 'X', 'V', '7', '9', 'B', 'P', 'H', 'Q', ']', ':', '>', '`', '#', '/', '!', '6', 'F', '0', 'C', 'G', '3', '.', '$', ';', 'A', 'J', '*', '<', ')', '4', '?', '_']


In [7]:
num_dic = {n: i for i, n in enumerate(char_sequence)}
dic_len = len(num_dic)
print(num_dic)

{'1': 0, 'R': 1, 'N': 2, '(': 3, ',': 4, 'I': 6, '%': 7, 'S': 8, 'K': 9, ';': 55, 'T': 10, 'W': 11, '[': 15, 'E': 13, '2': 14, '=': 17, 'Y': 19, "'": 20, '9': 35, '5': 21, '^': 22, '+': 24, ' ': 25, '*': 58, 'D': 5, '"': 26, '&': 28, 'U': 29, 'O': 30, 'M': 27, 'X': 32, 'V': 33, '7': 34, 'B': 36, 'J': 57, 'P': 37, 'Q': 39, ']': 40, ':': 41, '>': 42, '_': 63, '`': 43, '#': 44, '/': 45, 'H': 38, '6': 47, '0': 49, '?': 62, 'Z': 31, '8': 18, 'C': 50, 'G': 51, '.': 53, '$': 54, '\t': 16, 'A': 56, '3': 52, '<': 59, ')': 60, '4': 61, '!': 46, '-': 12, 'F': 48, 'L': 23}


In [8]:
input_max = len(max(input_lines, key=len).rstrip('\n'))
output_max = len(max(output_lines, key=len).rstrip('\n'))
print(input_max, '\n', output_max)

35 
 8


In [9]:
input_lines = [x.rstrip() + ('$' * (input_max - len(x.rstrip()))) for x in input_lines]
print(input_lines[0])
print(len(input_lines[0]))

SENSOR-AMBIENT$$$$$$$$$$$$$$$$$$$$$
35


In [10]:
output_lines = [x.rstrip() + ('$' * (output_max - len(x.rstrip()))) for x in output_lines]
print(output_lines[0])
print(len(output_lines[0]))

87 08 99
8


In [11]:
seq_data = [[input_lines[n].rstrip(), output_lines[n].rstrip()] for n in range(len(input_lines))]
print(seq_data[0:3])

[['SENSOR-AMBIENT$$$$$$$$$$$$$$$$$$$$$', '87 08 99'], ['SEAM TAPE$$$$$$$$$$$$$$$$$$$$$$$$$$', '87 08 99'], ['PACKING$$$$$$$$$$$$$$$$$$$$$$$$$$$$', '84 82 30']]


In [12]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값. 시작을 나타내는 ^ 심볼을 맨 앞에 붙여준다.
        output = [num_dic[n] for n in ('^' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 $ 를 붙인다.
        target = [num_dic[n] for n in (seq[1] + '$')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아님 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)

    return input_batch, output_batch, target_batch

### 옵션 설정

In [13]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 10000
# 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다.
n_class = n_input = dic_len
print(n_class)

64


### 신경망 모델 구성

In [14]:
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])

# 인코더 셀을 구성한다.
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)

# 디코더 셀을 구성한다.
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)


model = tf.layers.dense(outputs, n_class, activation=None)


cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


### 신경망 모델 학습

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

In [16]:
# check legnth
input_length_max = len(max(input_batch, key=len))
input_length_min = len(min(input_batch, key=len))
output_length_max = len(max(output_batch, key=len))
output_length_min = len(min(output_batch, key=len))
target_length_max = len(max(target_batch, key=len))
target_length_min = len(min(target_batch, key=len))

print('input:', input_length_min, '~', input_length_max)
print('output:', output_length_min, '~', output_length_max)
print('target:', target_length_min, '~', target_length_max)

input: 35 ~ 35
output: 9 ~ 9
target: 9 ~ 9


In [17]:
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 4.217771
Epoch: 0002 cost = 3.836066
Epoch: 0003 cost = 3.106123
Epoch: 0004 cost = 2.216317
Epoch: 0005 cost = 1.830537
Epoch: 0006 cost = 1.775324
Epoch: 0007 cost = 1.672383
Epoch: 0008 cost = 1.487542
Epoch: 0009 cost = 1.481903
Epoch: 0010 cost = 1.443937
Epoch: 0011 cost = 1.416108
Epoch: 0012 cost = 1.405992
Epoch: 0013 cost = 1.385112
Epoch: 0014 cost = 1.363927
Epoch: 0015 cost = 1.353872
Epoch: 0016 cost = 1.340324
Epoch: 0017 cost = 1.325336
Epoch: 0018 cost = 1.310039
Epoch: 0019 cost = 1.300789
Epoch: 0020 cost = 1.288448
Epoch: 0021 cost = 1.276379
Epoch: 0022 cost = 1.264588
Epoch: 0023 cost = 1.253821
Epoch: 0024 cost = 1.240895
Epoch: 0025 cost = 1.231983
Epoch: 0026 cost = 1.222787
Epoch: 0027 cost = 1.212608
Epoch: 0028 cost = 1.204572
Epoch: 0029 cost = 1.197569
Epoch: 0030 cost = 1.191494
Epoch: 0031 cost = 1.182463
Epoch: 0032 cost = 1.175694
Epoch: 0033 cost = 1.169599
Epoch: 0034 cost = 1.171253
Epoch: 0035 cost = 1.223454
Epoch: 0036 cost = 1

Epoch: 0294 cost = 0.790323
Epoch: 0295 cost = 0.793261
Epoch: 0296 cost = 0.789216
Epoch: 0297 cost = 0.790178
Epoch: 0298 cost = 0.788540
Epoch: 0299 cost = 0.787952
Epoch: 0300 cost = 0.787689
Epoch: 0301 cost = 0.787250
Epoch: 0302 cost = 0.785797
Epoch: 0303 cost = 0.785733
Epoch: 0304 cost = 0.784130
Epoch: 0305 cost = 0.783793
Epoch: 0306 cost = 0.783219
Epoch: 0307 cost = 0.782451
Epoch: 0308 cost = 0.781599
Epoch: 0309 cost = 0.782152
Epoch: 0310 cost = 0.782664
Epoch: 0311 cost = 0.783498
Epoch: 0312 cost = 0.789555
Epoch: 0313 cost = 0.799380
Epoch: 0314 cost = 0.784329
Epoch: 0315 cost = 0.787703
Epoch: 0316 cost = 0.798884
Epoch: 0317 cost = 0.781815
Epoch: 0318 cost = 0.794715
Epoch: 0319 cost = 0.792633
Epoch: 0320 cost = 0.792055
Epoch: 0321 cost = 0.790639
Epoch: 0322 cost = 0.788751
Epoch: 0323 cost = 0.782769
Epoch: 0324 cost = 0.785316
Epoch: 0325 cost = 0.786910
Epoch: 0326 cost = 0.791326
Epoch: 0327 cost = 0.812153
Epoch: 0328 cost = 0.810194
Epoch: 0329 cost = 0

Epoch: 0587 cost = 0.722250
Epoch: 0588 cost = 0.723096
Epoch: 0589 cost = 0.723157
Epoch: 0590 cost = 0.722784
Epoch: 0591 cost = 0.721248
Epoch: 0592 cost = 0.721758
Epoch: 0593 cost = 0.722463
Epoch: 0594 cost = 0.726649
Epoch: 0595 cost = 0.730718
Epoch: 0596 cost = 0.727952
Epoch: 0597 cost = 0.724956
Epoch: 0598 cost = 0.731494
Epoch: 0599 cost = 0.731100
Epoch: 0600 cost = 0.727705
Epoch: 0601 cost = 0.727586
Epoch: 0602 cost = 0.727254
Epoch: 0603 cost = 0.727448
Epoch: 0604 cost = 0.725091
Epoch: 0605 cost = 0.724368
Epoch: 0606 cost = 0.723257
Epoch: 0607 cost = 0.724325
Epoch: 0608 cost = 0.726712
Epoch: 0609 cost = 0.727132
Epoch: 0610 cost = 0.735010
Epoch: 0611 cost = 0.731856
Epoch: 0612 cost = 0.728113
Epoch: 0613 cost = 0.729512
Epoch: 0614 cost = 0.730262
Epoch: 0615 cost = 0.726110
Epoch: 0616 cost = 0.730291
Epoch: 0617 cost = 0.724352
Epoch: 0618 cost = 0.725476
Epoch: 0619 cost = 0.724108
Epoch: 0620 cost = 0.723763
Epoch: 0621 cost = 0.722783
Epoch: 0622 cost = 0

Epoch: 0880 cost = 0.709007
Epoch: 0881 cost = 0.711068
Epoch: 0882 cost = 0.720304
Epoch: 0883 cost = 0.714818
Epoch: 0884 cost = 0.713551
Epoch: 0885 cost = 0.709072
Epoch: 0886 cost = 0.710499
Epoch: 0887 cost = 0.710927
Epoch: 0888 cost = 0.708757
Epoch: 0889 cost = 0.708163
Epoch: 0890 cost = 0.710315
Epoch: 0891 cost = 0.708605
Epoch: 0892 cost = 0.706499
Epoch: 0893 cost = 0.707170
Epoch: 0894 cost = 0.706568
Epoch: 0895 cost = 0.707265
Epoch: 0896 cost = 0.708211
Epoch: 0897 cost = 0.707001
Epoch: 0898 cost = 0.706171
Epoch: 0899 cost = 0.704867
Epoch: 0900 cost = 0.705458
Epoch: 0901 cost = 0.705278
Epoch: 0902 cost = 0.704737
Epoch: 0903 cost = 0.704889
Epoch: 0904 cost = 0.704906
Epoch: 0905 cost = 0.705475
Epoch: 0906 cost = 0.710644
Epoch: 0907 cost = 0.717254
Epoch: 0908 cost = 0.717410
Epoch: 0909 cost = 0.711456
Epoch: 0910 cost = 0.708874
Epoch: 0911 cost = 0.723261
Epoch: 0912 cost = 0.730878
Epoch: 0913 cost = 0.713926
Epoch: 0914 cost = 0.723217
Epoch: 0915 cost = 0

Epoch: 1173 cost = 0.685375
Epoch: 1174 cost = 0.685405
Epoch: 1175 cost = 0.685081
Epoch: 1176 cost = 0.683996
Epoch: 1177 cost = 0.685605
Epoch: 1178 cost = 0.685096
Epoch: 1179 cost = 0.685147
Epoch: 1180 cost = 0.685332
Epoch: 1181 cost = 0.686737
Epoch: 1182 cost = 0.688124
Epoch: 1183 cost = 0.692891
Epoch: 1184 cost = 0.690433
Epoch: 1185 cost = 0.687555
Epoch: 1186 cost = 0.685891
Epoch: 1187 cost = 0.687346
Epoch: 1188 cost = 0.684964
Epoch: 1189 cost = 0.685329
Epoch: 1190 cost = 0.689954
Epoch: 1191 cost = 0.688993
Epoch: 1192 cost = 0.688651
Epoch: 1193 cost = 0.691393
Epoch: 1194 cost = 0.694981
Epoch: 1195 cost = 0.688239
Epoch: 1196 cost = 0.690081
Epoch: 1197 cost = 0.693057
Epoch: 1198 cost = 0.688557
Epoch: 1199 cost = 0.689849
Epoch: 1200 cost = 0.688506
Epoch: 1201 cost = 0.688131
Epoch: 1202 cost = 0.688286
Epoch: 1203 cost = 0.686921
Epoch: 1204 cost = 0.687517
Epoch: 1205 cost = 0.686939
Epoch: 1206 cost = 0.685604
Epoch: 1207 cost = 0.686101
Epoch: 1208 cost = 0

...

Epoch: 8791 cost = 0.724360
Epoch: 8792 cost = 0.728095
Epoch: 8793 cost = 0.725479
Epoch: 8794 cost = 0.726629
Epoch: 8795 cost = 0.725898
Epoch: 8796 cost = 0.724204
Epoch: 8797 cost = 0.726843
Epoch: 8798 cost = 0.724059
Epoch: 8799 cost = 0.725831
Epoch: 8800 cost = 0.723671
Epoch: 8801 cost = 0.724021
Epoch: 8802 cost = 0.724253
Epoch: 8803 cost = 0.723497
Epoch: 8804 cost = 0.722671
Epoch: 8805 cost = 0.722916
Epoch: 8806 cost = 0.722827
Epoch: 8807 cost = 0.723591
Epoch: 8808 cost = 0.722666
Epoch: 8809 cost = 0.722457
Epoch: 8810 cost = 0.721632
Epoch: 8811 cost = 0.722668
Epoch: 8812 cost = 0.722226
Epoch: 8813 cost = 0.722147
Epoch: 8814 cost = 0.722158
Epoch: 8815 cost = 0.721681
Epoch: 8816 cost = 0.721440
Epoch: 8817 cost = 0.721491
Epoch: 8818 cost = 0.721371
Epoch: 8819 cost = 0.721011
Epoch: 8820 cost = 0.721363
Epoch: 8821 cost = 0.721425
Epoch: 8822 cost = 0.720318
Epoch: 8823 cost = 0.721815
Epoch: 8824 cost = 0.720988
Epoch: 8825 cost = 0.721288
Epoch: 8826 cost = 0

Epoch: 9084 cost = 0.713960
Epoch: 9085 cost = 0.715810
Epoch: 9086 cost = 0.714936
Epoch: 9087 cost = 0.715726
Epoch: 9088 cost = 0.714859
Epoch: 9089 cost = 0.715663
Epoch: 9090 cost = 0.715098
Epoch: 9091 cost = 0.715581
Epoch: 9092 cost = 0.715614
Epoch: 9093 cost = 0.716288
Epoch: 9094 cost = 0.715084
Epoch: 9095 cost = 0.714928
Epoch: 9096 cost = 0.715377
Epoch: 9097 cost = 0.715073
Epoch: 9098 cost = 0.714663
Epoch: 9099 cost = 0.715686
Epoch: 9100 cost = 0.715306
Epoch: 9101 cost = 0.715037
Epoch: 9102 cost = 0.715110
Epoch: 9103 cost = 0.716341
Epoch: 9104 cost = 0.716968
Epoch: 9105 cost = 0.717493
Epoch: 9106 cost = 0.719878
Epoch: 9107 cost = 0.720782
Epoch: 9108 cost = 0.721287
Epoch: 9109 cost = 0.721059
Epoch: 9110 cost = 0.716032
Epoch: 9111 cost = 0.716291
Epoch: 9112 cost = 0.717237
Epoch: 9113 cost = 0.717590
Epoch: 9114 cost = 0.715460
Epoch: 9115 cost = 0.717168
Epoch: 9116 cost = 0.715775
Epoch: 9117 cost = 0.715877
Epoch: 9118 cost = 0.715483
Epoch: 9119 cost = 0

Epoch: 9377 cost = 0.711504
Epoch: 9378 cost = 0.710831
Epoch: 9379 cost = 0.711376
Epoch: 9380 cost = 0.711315
Epoch: 9381 cost = 0.711711
Epoch: 9382 cost = 0.711279
Epoch: 9383 cost = 0.712429
Epoch: 9384 cost = 0.711838
Epoch: 9385 cost = 0.713046
Epoch: 9386 cost = 0.712974
Epoch: 9387 cost = 0.712106
Epoch: 9388 cost = 0.714281
Epoch: 9389 cost = 0.714540
Epoch: 9390 cost = 0.713446
Epoch: 9391 cost = 0.715257
Epoch: 9392 cost = 0.715849
Epoch: 9393 cost = 0.716159
Epoch: 9394 cost = 0.716339
Epoch: 9395 cost = 0.714094
Epoch: 9396 cost = 0.713608
Epoch: 9397 cost = 0.714488
Epoch: 9398 cost = 0.714416
Epoch: 9399 cost = 0.714473
Epoch: 9400 cost = 0.714343
Epoch: 9401 cost = 0.715296
Epoch: 9402 cost = 0.715003
Epoch: 9403 cost = 0.714251
Epoch: 9404 cost = 0.714549
Epoch: 9405 cost = 0.714933
Epoch: 9406 cost = 0.713735
Epoch: 9407 cost = 0.712995
Epoch: 9408 cost = 0.713879
Epoch: 9409 cost = 0.713591
Epoch: 9410 cost = 0.713768
Epoch: 9411 cost = 0.712545
Epoch: 9412 cost = 0

Epoch: 9670 cost = 0.723387
Epoch: 9671 cost = 0.757962
Epoch: 9672 cost = 0.822123
Epoch: 9673 cost = 0.849010
Epoch: 9674 cost = 0.924207
Epoch: 9675 cost = 0.976951
Epoch: 9676 cost = 0.945486
Epoch: 9677 cost = 0.971379
Epoch: 9678 cost = 0.986698
Epoch: 9679 cost = 0.971889
Epoch: 9680 cost = 1.018322
Epoch: 9681 cost = 0.992603
Epoch: 9682 cost = 1.033910
Epoch: 9683 cost = 1.023249
Epoch: 9684 cost = 1.001313
Epoch: 9685 cost = 1.004429
Epoch: 9686 cost = 1.011165
Epoch: 9687 cost = 1.019188
Epoch: 9688 cost = 1.005322
Epoch: 9689 cost = 0.995376
Epoch: 9690 cost = 0.981820
Epoch: 9691 cost = 0.971047
Epoch: 9692 cost = 0.966607
Epoch: 9693 cost = 0.964042
Epoch: 9694 cost = 0.949771
Epoch: 9695 cost = 0.948240
Epoch: 9696 cost = 0.972759
Epoch: 9697 cost = 0.953422
Epoch: 9698 cost = 0.956034
Epoch: 9699 cost = 0.952894
Epoch: 9700 cost = 0.954091
Epoch: 9701 cost = 0.943114
Epoch: 9702 cost = 0.935472
Epoch: 9703 cost = 0.929544
Epoch: 9704 cost = 0.920530
Epoch: 9705 cost = 0

Epoch: 9963 cost = 0.766150
Epoch: 9964 cost = 0.766740
Epoch: 9965 cost = 0.766834
Epoch: 9966 cost = 0.765777
Epoch: 9967 cost = 0.765508
Epoch: 9968 cost = 0.765753
Epoch: 9969 cost = 0.765188
Epoch: 9970 cost = 0.765262
Epoch: 9971 cost = 0.764757
Epoch: 9972 cost = 0.765352
Epoch: 9973 cost = 0.764497
Epoch: 9974 cost = 0.765635
Epoch: 9975 cost = 0.764332
Epoch: 9976 cost = 0.763478
Epoch: 9977 cost = 0.764690
Epoch: 9978 cost = 0.763857
Epoch: 9979 cost = 0.764157
Epoch: 9980 cost = 0.764080
Epoch: 9981 cost = 0.763605
Epoch: 9982 cost = 0.764484
Epoch: 9983 cost = 0.763390
Epoch: 9984 cost = 0.762862
Epoch: 9985 cost = 0.763109
Epoch: 9986 cost = 0.763542
Epoch: 9987 cost = 0.762661
Epoch: 9988 cost = 0.762227
Epoch: 9989 cost = 0.763142
Epoch: 9990 cost = 0.763785
Epoch: 9991 cost = 0.762449
Epoch: 9992 cost = 0.761892
Epoch: 9993 cost = 0.762541
Epoch: 9994 cost = 0.762272
Epoch: 9995 cost = 0.763243
Epoch: 9996 cost = 0.762993
Epoch: 9997 cost = 0.766021
Epoch: 9998 cost = 0

### 번역 테스트

In [18]:
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
    # ['word', 'PPPP']
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    print(result[0])
    decoded = [char_sequence[i] for i in result[0]]

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    end = decoded.index('$')
    translated = ''.join(decoded[:end])

    return translated

In [21]:
print('SENSOR ->', translate('SLIDING MALE AND FEMALE H748'))

[35 35 25 49 47 25 35 49 54 18 49 25 52 61 35 54 18 18 25 18 52 25 35 35
 14 54 52 49 54]
SENSOR -> 99 06 90
